# Lesson 2

### Take Aways
* aaa
* bbb
* ccc

https://platform.openai.com/logs?api=traces

https://www.youtube.com/playlist?list=PLO66QfE8gWT0oM1hbfcFUa-2H3yI4vfg8

In [ ]:
# Install required packages

!pip install openai-agents


In [ ]:
import os
import warnings
from google.colab import userdata

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


In [ ]:
import requests
from typing import Dict, Optional
from dotenv import load_dotenv
import os
from agents import function_tool

load_dotenv()

# get api_key
api_key = userdata.get("WEATHER_API_KEY") os.getenv("weather_api_key")

@function_tool
def get_current_weather(location: str) -> Optional[Dict]:
    """
    Get current weather data from WeatherAPI.com

    Args:
        api_key (str): Your WeatherAPI API key
        location (str): Location name or coordinates (e.g., 'London' or '48.8567,2.3508')

    Returns:
        Optional[Dict]: Weather data dictionary or None if request fails
    """
    url = "https://api.weatherapi.com/v1/current.json"

    headers = {
        "Content-Type": "application/json"
    }

    params = {
        "key": api_key,
        "q": location
    }

    try:
        response = requests.post(url, headers=headers, params=params)
        response.raise_for_status()  # Raise exception for non-200 status codes
        if response:
            current = response.json().get('current', {})
            #print (f"Temperature: {current.get('temp_c')}°C")
            return (f"Temperature: {current.get('temp_c')}°C")

    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}")
        return None


#get_current_weather("New York")

In [ ]:
from agents import Agent, Runner
from tools import get_current_weather
from pydantic import BaseModel

class Weather(BaseModel):
    location : str
    temperature : float

# create agent
agent = Agent(
    name = "weather_agent",
    instructions = " You are an expert in providing weather information",
    tools = [get_current_weather],
    model = "gpt-4o-mini",
    output_type= Weather
)

# run the agent
async def main():
    result = await Runner.run(agent, "What is the weather in London?")
    print(result.final_output)

# Execute the asynchronous main function
await main()
